In [ ]:
# check if this instance of the notebook already has files present
# and thus determine which steps required prior to reading in file and handling the data
!ls

sample_data


In [ ]:
# set-up spark (NB if Apache amend versions on download site we will need to amend path in wget command)
## NOTE that this version would make use of Hadoop if installed BUT that HDFS & Hadoop is not installed on our Colab
## (we are only using a single node (probably as a VM) so we will not be able to benefit from parallelism)
!clear
!echo welcome

!rm -f spark-3.4.[01]-bin-hadoop3.tgz*
!rm -rf spark-3.4.[01]-bin-hadoop3

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz
!tar -xf spark-3.4.3-bin-hadoop3.tgz

!ls -alt
print("standalone Spark is now installed")

welcome
--2024-04-25 14:55:16--  https://downloads.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388930980 (371M) [application/x-gzip]
Saving to: ‘spark-3.4.3-bin-hadoop3.tgz’

spark-3.4.3-bin-had 100%[===================>] 370.91M  21.3MB/s    in 18s     

2024-04-25 14:55:35 (20.5 MB/s) - ‘spark-3.4.3-bin-hadoop3.tgz’ saved [388930980/388930980]

total 379840
drwxr-xr-x  1 root root      4096 Apr 25 14:55 .
drwxr-xr-x  1 root root      4096 Apr 25 14:54 ..
drwxr-xr-x  1 root root      4096 Apr 23 13:23 sample_data
drwxr-xr-x  4 root root      4096 Apr 23 13:22 .config
-rw-r--r--  1 root root 388930980 Apr 15 01:30 spark-3.4.3-bin-hadoop3.tgz
drwxr-xr-x 13 1000 1000      4096 Apr 15 01:28 spark-3.4.3-bin-hadoop3
standa

In [ ]:
# init spark (ensure SPARK_HOME set to same version as we download earlier)
!pip3 install findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
# the next line gives us 'local' mode. try 'local[2]' to use 2 cores or 'master:NNNN' to run on Spark standalone cluster at port NNNN
spark_conf = SparkConf().setMaster('local[2]').setAppName('MyApp')
sc = SparkContext(conf=spark_conf)
# see what we have by examining the Spark User Interface
from pyspark.sql import *
from pyspark.sql.functions import *
SparkSession.builder.getOrCreate()

In [ ]:
import os


# use 'spark' as reference (as per standard approach)
spark = SparkSession.builder.appName("bikes").getOrCreate()

In [ ]:
# get bike hire file for given year from TfL open data
!wget https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip
!unzip cyclehireusagestats-2014.zip

--2024-04-25 14:55:57--  https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip
Resolving cycling.data.tfl.gov.uk (cycling.data.tfl.gov.uk)... 104.16.98.104, 104.16.97.104
Connecting to cycling.data.tfl.gov.uk (cycling.data.tfl.gov.uk)|104.16.98.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225215129 (215M) [application/zip]
Saving to: ‘cyclehireusagestats-2014.zip’

cyclehireusagestats 100%[===================>] 214.78M   208MB/s    in 1.0s    

2024-04-25 14:55:58 (208 MB/s) - ‘cyclehireusagestats-2014.zip’ saved [225215129/225215129]

Archive:  cyclehireusagestats-2014.zip
  inflating: 1. Journey Data Extract 05Jan14-02Feb14.csv  
  inflating: 10a. Journey Data Extract 14Sep14-27Sep14.csv  
  inflating: 10b. Journey Data Extract 28Sep14-11Oct14.csv  
  inflating: 11a. Journey Data Extract 12Oct14-08Nov14.csv  
  inflating: 11b. Journey Data Extract 12Oct14-08Nov14.csv  
  inflating: 12a. Journey Data Extract 09Nov14-06Dec14.csv  
  i

In [ ]:
A="./1. Journey*csv"
B="./2. Journey*csv"
C="./3. Journey*csv"
D="./4. Journey*csv"
E="./5. Journey*csv"
F="./6. Journey*csv"
J="./7. Journey*csv"
H="./8a Journey*csv"
I="./8b Journey*csv"
L="./9a Journey*csv"
M="./9b Journey*csv"
N="./10a. Journey*csv"
O="./10b. Journey*csv"
P="./11a. Journey*csv"
Q="./11b. Journey*csv"
R="./12a. Journey*csv"
S="./12b. Journey*csv"
T="./13a. Journey*csv"
U="./13b. Journey*csv"
V="./14. Journey*csv"
spark = SparkSession.builder.appName("bikes").getOrCreate()
cycle = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load([A, B, C, D, E, F, H, I, J, L, M, N, O, P, Q, R, S, T, U, V]))

# show top 10
cycle.show(10)

+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+
|Rental Id|Duration|Bike Id|        End Date|EndStation Id|     EndStation Name|      Start Date|StartStation Id|   StartStation Name|
+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+
| 34263367|    1080|   9076|24/06/2014 00:57|          695|Islington Green, ...|24/06/2014 00:39|            311|Foley Street, Fit...|
| 34603487|     660|   6328|03/07/2014 11:51|          695|Islington Green, ...|03/07/2014 11:40|             22|Northington Stree...|
| 34689078|     120|   2006|05/07/2014 15:09|          357|Howland Street, F...|05/07/2014 15:07|            311|Foley Street, Fit...|
| 34724273|    1260|   7904|06/07/2014 16:35|          695|Islington Green, ...|06/07/2014 16:14|            311|Foley Street, Fit...|
| 34956750|    1500|   8251|13/07/2014 00:41|          

In [ ]:
from pyspark.sql import functions as F
from scipy.stats import ttest_ind, stats
from pyspark.sql.functions import to_timestamp
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, to_date, when
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import date_format
from scipy.stats import ttest_ind, stats
from pyspark.sql.functions import month
from pyspark.sql.functions import col

import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.functions import mean

In [ ]:

#Convert the date and time columns to timestamp format and calculate the duration of each ride.


cycle = cycle.withColumn('Start Date', to_timestamp('Start Date', 'dd/MM/yyyy HH:mm'))
cycle = cycle.withColumn('End Date', to_timestamp('End Date', 'dd/MM/yyyy HH:mm'))

cycle = cycle.withColumn('Duration', (cycle['End Date'].cast('long') - cycle['Start Date'].cast('long')))

In [ ]:
# Assuming 'cycle' is your PySpark DataFrame and it has a column named "Start Date" with date or timestamp type

# Initialize SparkSession (if not already initialized)
spark = SparkSession.builder.appName("year_extraction").getOrCreate()

# First, ensure the "Start Date" column is of the appropriate date or timestamp type
# If it's a string, you might need to convert it using to_date() or to_timestamp(), for example:
# cycle = cycle.withColumn("Start Date", to_date("Start Date", "your_date_format_here"))

# Extract the year from the "Start Date" column and get distinct values
lst_years = cycle.select(year("Start Date").alias("Year")).distinct().orderBy("Year")

# Show the list of years
lst_years.show()

+----+
|Year|
+----+
|null|
|2013|
|2014|
|2015|
+----+



In [ ]:
#drop the null values
cycle = cycle.drop(*[c for c in cycle.columns if cycle.select(c).na.drop().count() == 0])
cycle = cycle.dropna()

cycle.select([count(when(col(c).isNull(), c)).alias(c) for c in cycle.columns]).show()
cycle.show()

+---------+--------+-------+--------+-------------+---------------+----------+---------------+-----------------+
|Rental Id|Duration|Bike Id|End Date|EndStation Id|EndStation Name|Start Date|StartStation Id|StartStation Name|
+---------+--------+-------+--------+-------------+---------------+----------+---------------+-----------------+
|        0|       0|      0|       0|            0|              0|         0|              0|                0|
+---------+--------+-------+--------+-------------+---------------+----------+---------------+-----------------+

+---------+--------+-------+-------------------+-------------+--------------------+-------------------+---------------+--------------------+
|Rental Id|Duration|Bike Id|           End Date|EndStation Id|     EndStation Name|         Start Date|StartStation Id|   StartStation Name|
+---------+--------+-------+-------------------+-------------+--------------------+-------------------+---------------+--------------------+
| 34263367|

In [ ]:
# DataFrame with the filtered result where all durations are non-negative
cycle = cycle.filter(col("Duration") > 0)

In [ ]:
# removing year 2013 as the data we are working with starts from 2014
cycle = cycle.filter(year("Start Date") == 2014)

In [ ]:
#adding a column "day" representing the day of the week that corresponds to the start date
cycle = cycle.withColumn('Day', date_format('Start Date', 'EEEE'))
cycle.show()

+---------+--------+-------+-------------------+-------------+--------------------+-------------------+---------------+--------------------+---------+
|Rental Id|Duration|Bike Id|           End Date|EndStation Id|     EndStation Name|         Start Date|StartStation Id|   StartStation Name|      Day|
+---------+--------+-------+-------------------+-------------+--------------------+-------------------+---------------+--------------------+---------+
| 34263367|    1080|   9076|2014-06-24 00:57:00|          695|Islington Green, ...|2014-06-24 00:39:00|            311|Foley Street, Fit...|  Tuesday|
| 34603487|     660|   6328|2014-07-03 11:51:00|          695|Islington Green, ...|2014-07-03 11:40:00|             22|Northington Stree...| Thursday|
| 34689078|     120|   2006|2014-07-05 15:09:00|          357|Howland Street, F...|2014-07-05 15:07:00|            311|Foley Street, Fit...| Saturday|
| 34724273|    1260|   7904|2014-07-06 16:35:00|          695|Islington Green, ...|2014-07-06 

In [ ]:
# Count the number of distinct stations
total_stations = cycle.select(col("StartStation Name")).distinct().count()

print("Number of stations:", total_stations)

Number of stations: 761


In [ ]:
# Count the number of distinct stations
total_EndStation_Name = cycle.select(col("EndStation Name")).distinct().count()

print("EndStation Name:", total_EndStation_Name)

EndStation Name: 763


In [ ]:
# DataFrame with rides related to "Baylis Road" and "Waterloo station"


baylis_waterloo_rides = cycle.filter(cycle["StartStation Name"] == "Baylis Road, Waterloo")

baylis_waterloo_rides.show(10)

+---------+--------+-------+-------------------+-------------+--------------------+-------------------+---------------+--------------------+---------+
|Rental Id|Duration|Bike Id|           End Date|EndStation Id|     EndStation Name|         Start Date|StartStation Id|   StartStation Name|      Day|
+---------+--------+-------+-------------------+-------------+--------------------+-------------------+---------------+--------------------+---------+
| 34335942|     600|   7213|2014-06-25 18:59:00|          199|Great Tower Stree...|2014-06-25 18:49:00|            272|Baylis Road, Wate...|Wednesday|
| 34292680|     540|  11506|2014-06-24 18:12:00|          199|Great Tower Stree...|2014-06-24 18:03:00|            272|Baylis Road, Wate...|  Tuesday|
| 34251992|     600|   1845|2014-06-23 17:58:00|          199|Great Tower Stree...|2014-06-23 17:48:00|            272|Baylis Road, Wate...|   Monday|
| 34618956|     720|   8070|2014-07-03 17:59:00|          199|Great Tower Stree...|2014-07-03 

In [ ]:
other_rides = cycle.filter((cycle["StartStation Name"] != "Baylis Road, Waterloo"))
other_rides.show(10)

+---------+--------+-------+-------------------+-------------+--------------------+-------------------+---------------+--------------------+---------+
|Rental Id|Duration|Bike Id|           End Date|EndStation Id|     EndStation Name|         Start Date|StartStation Id|   StartStation Name|      Day|
+---------+--------+-------+-------------------+-------------+--------------------+-------------------+---------------+--------------------+---------+
| 34263367|    1080|   9076|2014-06-24 00:57:00|          695|Islington Green, ...|2014-06-24 00:39:00|            311|Foley Street, Fit...|  Tuesday|
| 34603487|     660|   6328|2014-07-03 11:51:00|          695|Islington Green, ...|2014-07-03 11:40:00|             22|Northington Stree...| Thursday|
| 34689078|     120|   2006|2014-07-05 15:09:00|          357|Howland Street, F...|2014-07-05 15:07:00|            311|Foley Street, Fit...| Saturday|
| 34724273|    1260|   7904|2014-07-06 16:35:00|          695|Islington Green, ...|2014-07-06 

In [ ]:
# Perform descriptive statistics
baylis_waterloo_stats = baylis_waterloo_rides.select(F.mean('Duration'), F.stddev('Duration')).collect()[0]
other_stats = other_rides.select(F.mean('Duration'), F.stddev('Duration')).collect()[0]

print("descriptive statistics:")
print("descriptive statistics_baylis:", baylis_waterloo_stats)
print("other_stats_other:", other_stats )

descriptive statistics:
descriptive statistics_baylis: Row(avg(Duration)=1046.6930031147915, stddev_samp(Duration)=6409.995032074472)
other_stats_other: Row(avg(Duration)=1470.5920967367313, stddev_samp(Duration)=13006.925132363982)


In [ ]:
# Collect the durations as lists
baylis_duration = baylis_waterloo_rides.select("Duration")
other_duration = other_rides.select("Duration")
baylis_waterloo_list = baylis_duration.rdd.flatMap(lambda x: x).collect()
other_rides_list = other_duration.rdd.flatMap(lambda x: x).collect()

In [ ]:
from scipy.stats import levene

# Perform Levene's test for equal variances
stat, p_value = levene(baylis_waterloo_list, other_rides_list)
print('Levene test p-value:', p_value)

# Set significance level
alpha = 0.05
if p_value < alpha:
    print("Unequal variances, reject the null hypothesis.")
else:
    print("Equal variances, fail to reject the null hypothesis.")


Levene test p-value: 9.830633262687688e-06
Unequal variances, reject the null hypothesis.


In [ ]:
# Perform a two-sample t-test

# Perform t-test on the duration data of bike rides
t_stat, p_value = ttest_ind(baylis_waterloo_list, other_rides_list, equal_var=False) # Added equal_var=False for Welch's t-test

# Print the resulting t-statistic and p-value
print("Results of Two-Sample T-Test:")
print("----------------------------")
print("T-Statistic: ", t_stat)
print("P-Value: ", p_value)

Results of Two-Sample T-Test:
----------------------------
T-Statistic:  -10.67130910576357
P-Value:  1.566891081171705e-26


In [ ]:
# Interpret the results
alpha = 0.05  # significance level
if p_value < alpha:
    print(f"Reject the null hypothesis - There is sufficient evidence to conclude that rides in 2014 starting from Baylis Road, Waterloo station were significantly shorter compared to other stations is true. P-value: {p_value}")
else:
    print(f"Fail to reject the null hypothesis - no significant difference found between two groups. P-value: {p_value}")

Reject the null hypothesis - There is sufficient evidence to conclude that rides in 2014 starting from Baylis Road, Waterloo station were significantly shorter compared to other stations is true. P-value: 1.566891081171705e-26


H1

In [ ]:
from pyspark.sql.functions import mean

# Calculate average duration for Baylis Road, Waterloo
baylis_avg_duration = baylis_waterloo_rides.agg(mean("Duration").alias("average_duration")).collect()[0]["average_duration"]

# Calculate average duration for other stations
other_avg_duration = other_rides.agg(mean("Duration").alias("average_duration")).collect()[0]["average_duration"]


In [ ]:
# Print results
print(f"Average duration from Baylis Road, Waterloo: {baylis_avg_duration} seconds")
print(f"Average duration from other stations: {other_avg_duration} seconds")

# Check if Baylis Road, Waterloo rides are shorter
if baylis_avg_duration < other_avg_duration:
    print("Rides starting from Baylis Road, Waterloo station are shorter than those starting from other stations.")
else:
    print("Rides starting from other stations are shorter than those starting from Baylis Road, Waterloo station.")



Average duration from Baylis Road, Waterloo: 1046.6930031147915 seconds
Average duration from other stations: 1470.5920967367313 seconds
Rides starting from Baylis Road, Waterloo station are shorter than those starting from other stations.
